
# Global Cost of Living & Affordability — EDA

This project analyzes the global cost of living using The Economist’s Big Mac Index combined with World Bank indicators.

The goal is to explore how Big Mac prices relate to GDP per capita, inflation, unemployment, and income inequality across different countries and regions.
Through data cleaning, visualization, and simple modeling, the project highlights affordability patterns and global economic differences.


#Project Summary
Data Sources:

Big Mac Index (The Economist, 2000–latest year)

World Bank Indicators via wbgapi

Dataset Size: ~1,900 rows × 21 features

Countries: 73

Indicators Used: GDP per capita, inflation, unemployment, urban %, population, and Gini index


#Key Steps
Data Cleaning & Preparation

Loaded data directly from The Economist’s GitHub and the World Bank API.

Standardized dates, country codes, and currency fields.

Merged both datasets by ISO3 and year.

Filled missing values within each country.

Feature Engineering

Visualization

Statistical Modeling


#Results
Global Mean Big Mac Price: $4.96

Median Price: $3.69

Average Price Growth (2000–2022): ~3.6% per year

Correlation (log price vs log GDP): 0.20 (weak positive)

Most Volatile Prices: Lebanon, Venezuela, Argentina

93% of countries have Big Macs cheaper than the U.S.



#Summary

This project connects global economics and data visualization in an approachable way.

By analyzing burger prices alongside World Bank indicators, it highlights how affordability varies around the world — showing that something as simple as a Big Mac can reveal real economic differences in income, inflation, and purchasing power.

#Import All Necessary Items

In [ ]:
%pip install wbgapi

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import wbgapi as wb


import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import wbgapi as wb
from pathlib import Path

# Optional but nice: consistent styling
pio.templates.default = "plotly_white"
pd.set_option("display.max_columns", 80)

# Folders
BASE = Path.cwd()
DATA_DIR = BASE / "data"
FIG_DIR  = BASE / "figures"
DATA_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)

def save_fig(fig, name: str):
    """
    Save figure as PNG (if Kaleido is present) and always as HTML.
    """
    try:
        fig.write_image(str(FIG_DIR / f"{name}.png"))
    except Exception:
        pass
    fig.write_html(str(FIG_DIR / f"{name}.html"), include_plotlyjs="cdn", full_html=False)
    print(f"   ✓ Saved figures/{name}.html" + (" and .png" if (FIG_DIR / f'{name}.png').exists() else ""))

def first_col(df: pd.DataFrame, options):
    """Return existing column matching any of options (case-insensitive)."""
    low = {c.lower(): c for c in df.columns}
    for opt in options:
        if opt.lower() in low:
            return low[opt.lower()]
    raise KeyError(f"None of {options} found in columns: {list(df.columns)}")

print("=" * 70)
print("GLOBAL COST OF LIVING ANALYSIS: Big Mac Index & Economic Indicators")
print("=" * 70)

GLOBAL COST OF LIVING ANALYSIS: Big Mac Index & Economic Indicators


#Loading in Big Mac Data

In [ ]:
print("STEP 1: Loading Big Mac Index data")

# URL from The Economist's GitHub
BIG_MAC_URL = "https://raw.githubusercontent.com/TheEconomist/big-mac-data/master/source-data/big-mac-source-data.csv"

bm = pd.read_csv(BIG_MAC_URL)
print(f"   → Raw data loaded: {bm.shape}")
print(f"   → Available columns: {list(bm.columns)}")

# Process based on actual columns
bm['date'] = pd.to_datetime(bm['date'])
bm['year'] = bm['date'].dt.year

# Create standardized column names based on what exists
bm = bm.copy()

# ISO code
if 'iso_a3' in bm.columns:
    bm['iso3'] = bm['iso_a3'].str.upper()
elif 'ISO' in bm.columns:
    bm['iso3'] = bm['ISO'].str.upper()

# Country name
if 'name' in bm.columns:
    bm['country'] = bm['name']

# USD price - try multiple column names
if 'dollar_price' in bm.columns:
    bm['usd_price'] = bm['dollar_price']
elif 'USD' in bm.columns:
    bm['usd_price'] = bm['USD']
elif 'usd' in bm.columns:
    bm['usd_price'] = bm['usd']
else:
    # If no direct USD column, look for calculation method
    if 'local_price' in bm.columns and 'dollar_ex' in bm.columns:
        bm['usd_price'] = bm['local_price'] / bm['dollar_ex']
        print("   → Calculated usd_price from local_price / dollar_ex")

# Verify we have required columns
required = ['iso3', 'country', 'date', 'year', 'usd_price']
missing = [col for col in required if col not in bm.columns]
if missing:
    print(f"   ✗ ERROR: Missing columns: {missing}")
    print(f"   Available: {list(bm.columns)}")
    raise ValueError(f"Cannot proceed without: {missing}")

# Select only needed columns
keep_cols = ['iso3', 'country', 'date', 'year', 'usd_price']
if 'local_price' in bm.columns:
    keep_cols.append('local_price')

bm = bm[keep_cols].dropna(subset=['iso3', 'country', 'usd_price'])

print(f"   ✓ Countries: {bm['iso3'].nunique()}")
print(f"   ✓ Time range: {bm['year'].min()} - {bm['year'].max()}")
print(f"   ✓ Total observations: {len(bm):,}")

STEP 1: Loading Big Mac Index data
   → Raw data loaded: (1948, 7)
   → Available columns: ['name', 'iso_a3', 'currency_code', 'local_price', 'dollar_ex', 'GDP_dollar', 'date']
   → Calculated usd_price from local_price / dollar_ex
   ✓ Countries: 73
   ✓ Time range: 2000 - 2022
   ✓ Total observations: 1,947


#Loading in World Bank

In [ ]:
bm["year"] = pd.to_datetime(bm["date"]).dt.year.astype(int)

bm = bm[["iso3","country","date","year","usd_price","local_price"]]
print(f"   ✓ Countries: {bm['iso3'].nunique()} | Years: {bm['year'].min()}–{bm['year'].max()} | Rows: {len(bm):,}")

# ============================= STEP 2: WORLD BANK INDICATORS =============================
print("STEP 2: Load World Bank indicators")

# Recreate year defensively in case user runs cells out of order
if "year" not in bm.columns:
    bm["year"] = pd.to_datetime(bm["date"]).dt.year.astype(int)

years_start = int(bm["year"].min())
years_end   = int(bm["year"].max())
years = range(years_start, years_end + 1)

INDICATORS = {
    "NY.GDP.PCAP.PP.CD": "gdp_ppp_pc",
    "FP.CPI.TOTL.ZG":    "inflation_pct",
    "SL.UEM.TOTL.ZS":    "unemployment_pct",
    "SP.URB.TOTL.IN.ZS": "urban_pct",
    "SP.POP.TOTL":       "population",
    "SI.POV.GINI":       "gini_index",
}

wb_raw = wb.data.DataFrame(list(INDICATORS.keys()), time=years, labels=True).reset_index()
wb_df = wb_raw.rename(columns={"time":"year","economy":"iso3"})

# Rename YRxxxx columns to year
year_cols = {col: int(col[2:]) for col in wb_df.columns if col.startswith('YR')}
wb_df = wb_df.rename(columns=year_cols)

# Melt the DataFrame to have a single year column
wb_df = wb_df.melt(id_vars=['iso3', 'series', 'Country', 'Series'],
                   value_vars=year_cols.values(),
                   var_name='year', value_name='value')

# Pivot the DataFrame to have indicators as columns
wb_df = wb_df.pivot_table(index=['iso3', 'year'], columns='series', values='value').reset_index()

# Rename indicator columns
for old,new in INDICATORS.items():
    if old in wb_df.columns: wb_df = wb_df.rename(columns={old:new})

wb_df["year"] = wb_df["year"].astype(int)

# Region & income group
meta = wb.economy.DataFrame().reset_index().rename(columns={"id":"iso3"})
def _clean_meta(v):
    s = str(v)
    if "value" in s:
        try: return s.split("value': '",1)[1].split("'}",1)[0]
        except Exception: return None
    return None
meta["region"] = meta["region"].map(_clean_meta)
meta["income_group"] = meta["incomeLevel"].map(_clean_meta)

wb_df = wb_df.merge(meta[["iso3","region","income_group"]], on="iso3", how="left")
print("   ✓ WB frame:", wb_df.shape)

   ✓ Countries: 73 | Years: 2000–2022 | Rows: 1,947
STEP 2: Load World Bank indicators


#Merging Datasets

In [ ]:
print("STEP 3: Merging datasets and engineering features")

df = bm.merge(wb_df, on=["iso3", "year"], how="left")

# Fill missing macro values within each country (forward/back)
for col in ["gdp_ppp_pc", "inflation_pct", "unemployment_pct", "urban_pct", "gini_index", "population"]:
    if col in df.columns:
        df[col] = df.groupby("iso3")[col].transform(lambda s: s.ffill().bfill())

# Derived metrics
df["daily_gdp"]     = df["gdp_ppp_pc"] / 365.0
df["afford_ratio"]  = df["usd_price"] / df["daily_gdp"]         # burger / daily GDP per capita
df["log_price"]     = np.log(df["usd_price"])
df["log_gdp"]       = np.log(df["gdp_ppp_pc"])

# Relative to USA price on the same date
us_prices = df.loc[df["iso3"].eq("USA"), ["date", "usd_price"]].rename(columns={"usd_price": "us_price"})
df = df.merge(us_prices, on="date", how="left")
df["rel_to_us"] = df["usd_price"] / df["us_price"]

# Income quartiles (drop duplicates to avoid warnings with identical bins)
df["income_quartile"] = pd.qcut(df["gdp_ppp_pc"], q=4,
                                labels=["Q1-Low", "Q2-Mid-Low", "Q3-Mid-High", "Q4-High"],
                                duplicates="drop")

# Latest snapshot per country
latest = df.loc[df.groupby("iso3")["date"].idxmax()].copy()

# Save dataset
output_file = DATA_DIR / "processed_cost_of_living.csv"
df.to_csv(output_file, index=False)

print(f"   ✓ Final shape: {df.shape}")
print(f"   ✓ Countries: {df['iso3'].nunique()}")
print(f"   ✓ Saved to {output_file}")



STEP 3: Merging datasets and engineering features
   ✓ Final shape: (1947, 21)
   ✓ Countries: 73
   ✓ Saved to /content/data/processed_cost_of_living.csv


#Visualizations

In [40]:
print("STEP 4: Visualizations (patched)")

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# ensure plots render in notebook
if not pio.renderers.default:
    pio.renderers.default = "notebook_connected"

def try_save(fig, name):
    try:
        save_fig(fig, name)
    except Exception as e:
        print(f"[save skipped for {name}] {e}")

def has_cols(df, cols):
    return all(c in df.columns for c in cols)

# ---------- ensure required columns / frames exist ----------
if "year" not in df.columns and "date" in df.columns:
    df["year"] = pd.to_datetime(df["date"]).dt.year

if "latest" not in globals() and "latest" not in locals():
    if "date" in df.columns:
        _maxd = pd.to_datetime(df["date"]).max()
        latest = df[pd.to_datetime(df["date"]) == _maxd].copy()
    else:
        _maxy = int(df["year"].max())
        latest = df[df["year"] == _maxy].copy()

if latest.empty:
    raise RuntimeError("Latest snapshot is empty after merge; cannot plot.")

# ---------- harden types so plots don't go empty ----------
num_cols = ["usd_price","gdp_ppp_pc","inflation_pct","unemployment_pct",
            "urban_pct","gini_index","population","afford_ratio","rel_to_us"]
for c in [c for c in num_cols if c in latest.columns]:
    latest[c] = pd.to_numeric(latest[c], errors="coerce")

# Sometimes country is called 'name'; backfill if needed
if "country" not in latest.columns and "name" in latest.columns:
    latest["country"] = latest["name"]



# =====================================================================
# 1) GDP vs PRICE SCATTER
# =====================================================================
# ---------- 2) GDP vs PRICE SCATTER (Y-axis limited to 10 USD) ----------
need_cols = ["gdp_ppp_pc", "usd_price", "country"]
if has_cols(latest, need_cols):
    scatter_data = latest.dropna(subset=["gdp_ppp_pc", "usd_price", "country"])
    if not scatter_data.empty:
        fig4 = px.scatter(
            scatter_data,
            x="gdp_ppp_pc",
            y="usd_price",
            hover_name="country",
            color="region" if "region" in scatter_data.columns else None,
            size="population" if "population" in scatter_data.columns else None,
            trendline="ols",
            title="GDP per Capita (PPP) vs Big Mac Price (Latest)",
            labels={
                "gdp_ppp_pc": "GDP per capita, PPP (current int'l $)",
                "usd_price": "Price (USD)"
            }
        )

        # Limit Y-axis to 10 USD for better readability
        fig4.update_yaxes(range=[0, 10])

        try_save(fig4, "04_gdp_vs_price_scatter")
        fig4.show()


# =====================================================================
# 2) CHOROPLETH RELATIVE TO US
# =====================================================================
if has_cols(latest, ["iso3", "rel_to_us", "country"]):
    map_data = latest.dropna(subset=["rel_to_us", "iso3"]).copy()

    if not map_data.empty:
        import numpy as np
        import pandas as pd
        import plotly.express as px

        # ======= choose your bin size here =======
        BIN_STEP = 0.2
        # =========================================

        # Expand the range slightly so edge values show up and the legend is stable
        data_min, data_max = map_data["rel_to_us"].min(), map_data["rel_to_us"].max()
        vmin = np.floor((min(0.5, data_min)) / BIN_STEP) * BIN_STEP
        vmax = np.ceil((max(2.0, data_max)) / BIN_STEP) * BIN_STEP

        # Create bin edges and labels
        edges = np.arange(vmin, vmax + BIN_STEP, BIN_STEP)
        # Avoid zero-length if data is super narrow
        if len(edges) < 3:
            edges = np.array([vmin, vmin + BIN_STEP, vmin + 2*BIN_STEP])

        # Bin and create readable labels (e.g., "0.80–0.90", "0.90–1.00", ...)
        bins = pd.cut(
            map_data["rel_to_us"].clip(vmin, vmax),
            bins=edges,
            right=False,           # left-closed, right-open: [a, b)
            include_lowest=True
        )
        labels = [f"{left:.2f}–{right:.2f}" for left, right in zip(edges[:-1], edges[1:])]
        # Ensure consistent category order in legend
        bins = bins.cat.set_categories(pd.IntervalIndex.from_breaks(edges, closed="left"), ordered=True)
        map_data["rel_bin"] = bins.astype(str)  # use string labels for discrete legend

        # Build a discrete color list, one per bin (Turbo is vivid; try 'Viridis'/'Plasma' if you prefer)
        n_bins = len(edges) - 1
        # sample evenly across a colorscale to get n discrete colors
        stops = [i / max(n_bins - 1, 1) for i in range(n_bins)]
        color_list = px.colors.sample_colorscale("Turbo", stops)

        fig5 = px.choropleth(
            map_data,
            locations="iso3",
            color="rel_bin",                        # categorical → discrete legend
            hover_name="country",
            color_discrete_sequence=color_list,
            category_orders={"rel_bin": [str(iv) for iv in bins.cat.categories]},
            title="Big Mac Price Relative to the United States (Latest)",
        )

        # Better hover: show the actual numeric ratio too
        fig5.update_traces(
            hovertemplate="<b>%{hovertext}</b><br>Range: %{customdata[0]}<br>Ratio: %{customdata[1]:.2f}<extra></extra>",
            customdata=np.stack([map_data["rel_bin"], map_data["rel_to_us"]], axis=1)
        )

        # Styling
        fig5.update_layout(
            legend_title_text="Price Ratio vs US",
            geo=dict(showframe=False, showcoastlines=True, projection_type="natural earth"),
            margin=dict(l=10, r=10, t=50, b=10)
        )

        try_save(fig5, "05_world_map_discrete")
        fig5.show()

# =====================================================================
# 3) TIME TRENDS
# =====================================================================
if "year" in df.columns and "usd_price" in df.columns:
    agg_cols = {"usd_price":"mean"}
    if "afford_ratio" in df.columns:
        agg_cols["afford_ratio"] = "mean"
    yearly = df.groupby("year", as_index=False).agg(agg_cols)
    fig6 = make_subplots(specs=[[{"secondary_y": True}]])
    fig6.add_trace(go.Scatter(x=yearly["year"], y=yearly["usd_price"],
                              name="Avg Price (USD)", mode="lines+markers"),
                   secondary_y=False)
    if "afford_ratio" in yearly.columns:
        fig6.add_trace(go.Scatter(x=yearly["year"], y=yearly["afford_ratio"],
                                  name="Avg Affordability", mode="lines+markers"),
                       secondary_y=True)
        fig6.update_yaxes(title_text="Affordability Ratio", secondary_y=True)
    fig6.update_layout(title="Global Trends Over Time")
    fig6.update_xaxes(title_text="Year")
    fig6.update_yaxes(title_text="Price (USD)", secondary_y=False)
    try_save(fig6, "06_time_trends")
    fig6.show()

# =====================================================================
# 4) CORRELATION HEATMAP (cleaned + centered scale)
# =====================================================================
corr_vars_all = ["usd_price","gdp_ppp_pc","inflation_pct","unemployment_pct",
                 "urban_pct","gini_index","afford_ratio"]

available = [c for c in corr_vars_all if c in latest.columns]
if len(available) >= 2:
    # (Optional) drop almost-duplicate columns so colors aren't dominated
    dedup = available.copy()
    if set(["usd_price","afford_ratio"]).issubset(dedup):
        dedup.remove("afford_ratio")  # it's ~perfectly correlated with usd_price by design

    corr_df = latest[dedup].apply(pd.to_numeric, errors="coerce")
    corr_df = corr_df.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    # remove zero-variance cols
    keep = [c for c in corr_df.columns if corr_df[c].count() >= 3 and corr_df[c].std(skipna=True) > 0]
    corr_df = corr_df[keep]

    if len(keep) >= 2:
        cm = corr_df.corr(numeric_only=True)
        fig7 = px.imshow(
            cm.round(2),
            x=cm.columns, y=cm.index,
            text_auto=".2f", aspect="auto",
            zmin=-1, zmax=1,  # removed zmid
            color_continuous_scale="RdBu_r",
            title="Correlation Matrix (Latest Snapshot)"
        )
        fig7.update_xaxes(side="top")
        try_save(fig7, "07_correlation_heatmap")
        fig7.show()
    else:
        print("   ⚠️ Heatmap skipped: not enough variable variation after cleaning.")
else:
    print("   ⚠️ Heatmap skipped: fewer than 2 eligible variables.")



STEP 4: Visualizations (patched)
   ✓ Saved figures/04_gdp_vs_price_scatter.html


   ✓ Saved figures/05_world_map_discrete.html


   ✓ Saved figures/06_time_trends.html


   ✓ Saved figures/07_correlation_heatmap.html


#Regression Analysis

In [ ]:
print("STEP 5: Running regression analysis")

# OLS Regression with robust standard errors
reg_data = latest.dropna(subset=['log_price', 'log_gdp', 'inflation_pct', 'unemployment_pct', 'urban_pct'])
X = reg_data[['log_gdp', 'inflation_pct', 'unemployment_pct', 'urban_pct']]
X = sm.add_constant(X)
y = reg_data['log_price']

model = sm.OLS(y, X).fit(cov_type='HC3')  # HC3 = robust to heteroskedasticity
print(model.summary())

# Save regression output
with open(DATA_DIR / "regression_results.txt", 'w') as f:
    f.write(model.summary().as_text())
print(f"   ✓ Saved regression output")


STEP 5: Running regression analysis
                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     7.199
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           7.13e-05
Time:                        19:41:05   Log-Likelihood:                -41.305
No. Observations:                  71   AIC:                             92.61
Df Residuals:                      66   BIC:                             103.9
Df Model:                           4                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
cons

In [ ]:
print("STEP 6: K-Means clustering")

cluster_vars = [c for c in ["usd_price","gdp_ppp_pc","inflation_pct","unemployment_pct","gini_index"] if c in latest.columns]
cd = latest[["iso3","country"] + cluster_vars].dropna()

if len(cd) >= 10 and len(cluster_vars) >= 2:
    scaler = StandardScaler()
    Xs = scaler.fit_transform(cd[cluster_vars])
    n_clusters = max(2, min(4, len(cd)//3))
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cd["cluster"] = kmeans.fit_predict(Xs)
    fig9 = px.scatter(cd, x="gdp_ppp_pc", y="usd_price",
                      color="cluster", hover_name="country",
                      title="Country Clusters (K-Means)",
                      labels={"gdp_ppp_pc":"GDP per capita (PPP)","usd_price":"Price (USD)"})
    save_fig(fig9, "09_country_clusters")
else:
    print("   ⚠️ Not enough complete rows for clustering; skipped.")


STEP 6: K-Means clustering
   ✓ Saved figures/09_country_clusters.html


#Questions

In [ ]:
# Q1 — What are the global mean/median Big Mac prices in the latest snapshot, and a 95% CI for the mean?
print("\nQ1) Global mean, median, and 95% CI for latest Big Mac price (USD)")
from statsmodels.stats.weightstats import DescrStatsW

_ = latest.dropna(subset=["usd_price"]).copy()
ds = DescrStatsW(_["usd_price"].values)
mean_ = ds.mean
ci_low, ci_high = ds.tconfint_mean(alpha=0.05)
median_ = _["usd_price"].median()
n_ = len(_)
print(f"  n={n_}, mean=${mean_:.2f}, median=${median_:.2f}, 95% CI=(${ci_low:.2f}, ${ci_high:.2f})")



Q1) Global mean, median, and 95% CI for latest Big Mac price (USD)
  n=73, mean=$4.96, median=$3.69, 95% CI=($2.70, $7.22)


In [ ]:
# Q2 — What is the correlation between log price and log GDP per capita? Also print simple correlations for other macros.
print("\nQ2) Correlations : log(price) with log(GDP per capita) and other macros")
cols = [c for c in ["log_price","log_gdp","inflation_pct","unemployment_pct","urban_pct","gini_index"] if c in latest.columns]
corr = latest[cols].dropna().corr().round(3)
print(corr.to_string())
print(f"  Corr(log_price, log_gdp) = {corr.loc['log_price','log_gdp']:.3f}")


Q2) Correlations : log(price) with log(GDP per capita) and other macros
                  log_price  log_gdp  inflation_pct  unemployment_pct  urban_pct  gini_index
log_price             1.000    0.199          0.229             0.060      0.348      -0.099
log_gdp               0.199    1.000         -0.254            -0.220      0.507      -0.382
inflation_pct         0.229   -0.254          1.000             0.108      0.125       0.169
unemployment_pct      0.060   -0.220          0.108             1.000      0.126       0.566
urban_pct             0.348    0.507          0.125             0.126      1.000       0.172
gini_index           -0.099   -0.382          0.169             0.566      0.172       1.000
  Corr(log_price, log_gdp) = 0.199


In [ ]:
# Q3 — Are prices higher in high-income countries than in low-income countries?
print("\nQ3) Welch t-test: mean price in Q4-High vs Q1-Low income groups ")
from statsmodels.stats.weightstats import ttest_ind

sub = latest.dropna(subset=["usd_price","income_quartile"]).copy()
A = sub.loc[sub["income_quartile"].eq("Q4-High"), "usd_price"].values
B = sub.loc[sub["income_quartile"].eq("Q1-Low"),  "usd_price"].values
if len(A) >= 5 and len(B) >= 5:
    t, p, dfree = ttest_ind(A, B, usevar='unequal')
    print(f"  n_high={len(A)}, mean_high=${A.mean():.2f}")
    print(f"  n_low ={len(B)}, mean_low =$${B.mean():.2f}")
    print(f"  Welch t={t:.3f}, df≈{dfree:.1f}, p-value={p:.4f}")


Q3) Welch t-test: mean price in Q4-High vs Q1-Low income groups 
  n_high=32, mean_high=$4.52
  n_low =11, mean_low =$$10.67
  Welch t=-0.816, df≈10.0, p-value=0.4335


In [ ]:
# Q4 — What share of countries have Big Mac prices below the U.S., and what is the average multiple vs. the U.S.?
print("\nQ4) Share below U.S. price and average price multiple vs U.S. ")
q = latest.dropna(subset=["rel_to_us"]).copy()
share_below = (q["rel_to_us"] < 1).mean()
avg_multiple = q["rel_to_us"].mean()
print(f"  Countries with price < U.S.: {share_below*100:.1f}%")
print(f"  Average price multiple vs U.S.: {avg_multiple:.3f}×")


Q4) Share below U.S. price and average price multiple vs U.S. 
  Countries with price < U.S.: 93.2%
  Average price multiple vs U.S.: 0.860×


In [ ]:
# Q5 — What is the global annualized price growth (CAGR) from the first to last year available?
print("\nQ5) Global CAGR of average price from first to last year")
yearly = df.groupby("year", as_index=False)["usd_price"].mean().dropna()
p0, pT = yearly["usd_price"].iloc[0], yearly["usd_price"].iloc[-1]
n_years = int(yearly["year"].iloc[-1] - yearly["year"].iloc[0])
cagr = (pT/p0)**(1/max(n_years,1)) - 1 if p0>0 else np.nan
print(f"  {yearly['year'].iloc[0]} → {yearly['year'].iloc[-1]}: mean price ${p0:.2f} → ${pT:.2f}")
print(f"  CAGR = {cagr*100:.2f}% per year over {n_years} years")


Q5) Global CAGR of average price from first to last year
  2000 → 2022: mean price $2.06 → $4.45
  CAGR = 3.57% per year over 22 years


In [ ]:
# Q6 — Which 5 countries are most volatile by the std. dev. of YoY log price changes?
print("\nQ6) Top-5 most volatile countries by std dev of Δlog(price) YoY")
def _yoy_vol(g):
    g = g.sort_values("year")
    lp = np.log(g["usd_price"]).replace([np.inf,-np.inf], np.nan).dropna()
    dlp = lp.diff().dropna()
    return dlp.std() if len(dlp)>0 else np.nan

vol = df.groupby(["iso3","country"]).apply(_yoy_vol).reset_index(name="volatility_std_dlog").dropna()
top5 = vol.sort_values("volatility_std_dlog", ascending=False).head(5)
print(top5.to_string(index=False, formatters={"volatility_std_dlog":lambda x: f"{x:.3f}"}))
(top5).to_csv(DATA_DIR/"Q6_top5_volatility.csv", index=False)


Q6) Top-5 most volatile countries by std dev of Δlog(price) YoY
iso3      country volatility_std_dlog
 LBN      Lebanon               1.288
 VEN    Venezuela               0.615
 ARG    Argentina               0.317
 SAU Saudi Arabia               0.239
 RUS       Russia               0.171


In [ ]:
# Q7 — Simple OLS: log_price ~ log_gdp
print("\nQ7) OLS: log_price ~ log_gdp ")
import statsmodels.api as sm
sub = latest.dropna(subset=["log_price","log_gdp"]).copy()
X = sm.add_constant(sub[["log_gdp"]])
y = sub["log_price"]
model_simple = sm.OLS(y, X).fit()
b0, b1 = model_simple.params["const"], model_simple.params["log_gdp"]
print(f"  n={int(model_simple.nobs)}, beta0={b0:.3f}, beta_log_gdp={b1:.3f}, R^2={model_simple.rsquared:.3f}, AIC={model_simple.aic:.2f}")


Q7) OLS: log_price ~ log_gdp 
  n=71, beta0=0.087, beta_log_gdp=0.120, R^2=0.037, AIC=95.52


In [ ]:
# Q8 — Train/test performance (sklearn metrics)
print("\nQ8) Train/test RMSE & MAE for predicting price from macros ")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

feat = ["log_gdp","inflation_pct","unemployment_pct","urban_pct"]
tt = latest.dropna(subset=["usd_price"]+feat).copy()
X = tt[feat].values
y = tt["usd_price"].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=42)

lr = LinearRegression().fit(X_tr, y_tr)
pred = lr.predict(X_te)
mse = mean_squared_error(y_te, pred) # Get MSE
rmse = np.sqrt(mse) # Calculate RMSE
mae  = mean_absolute_error(y_te, pred)
print(f"  Test RMSE=${rmse:.3f}, MAE=${mae:.3f}, n_train={len(y_tr)}, n_test={len(y_te)}")


Q8) Train/test RMSE & MAE for predicting price from macros 
  Test RMSE=$2.190, MAE=$1.576, n_train=53, n_test=18


In [ ]:
# Q9 — Inflation pass-through in changes
print("\nQ9) Δlog(price) on same-year inflation (pooled); report coefficient and p-value")
d = df.sort_values(["iso3","year"]).copy()
d["log_price"]  = np.log(d["usd_price"])
d["dlog_price"] = d.groupby("iso3")["log_price"].diff()
reg = d.dropna(subset=["dlog_price","inflation_pct"]).copy()

X = sm.add_constant(reg[["inflation_pct"]])
y = reg["dlog_price"]
m_pass = sm.OLS(y, X).fit(cov_type="HC3")
coef = m_pass.params["inflation_pct"]
pval = m_pass.pvalues["inflation_pct"]
print(f"  β̂_inflation = {coef:.4f}, p-value = {pval:.4f}, n={int(m_pass.nobs)}")



Q9) Δlog(price) on same-year inflation (pooled); report coefficient and p-value
  β̂_inflation = 0.0006, p-value = 0.6074, n=1802


In [ ]:
# Q10 — Lag-1 autocorrelation of global mean price
print("\nQ10) Lag-1 autocorrelation of yearly global mean price")
yr = df.groupby("year", as_index=False)["usd_price"].mean().dropna()
x = yr["usd_price"].values
ac1 = np.corrcoef(x[1:], x[:-1])[0,1] if len(x) > 2 else np.nan
print(f"  ρ(1) = {ac1:.3f} (years {int(yr['year'].min())}–{int(yr['year'].max())})")


Q10) Lag-1 autocorrelation of yearly global mean price
  ρ(1) = 0.918 (years 2000–2022)


In [ ]:
# Q11 — Worst affordability ranking (latest)
print("\nQ11) Top-10 worst affordability (highest burger / daily GDPpc)")
tab = latest.dropna(subset=["afford_ratio"])[["country","afford_ratio"]].sort_values("afford_ratio", ascending=False).head(10)
print(tab.to_string(index=False, formatters={"afford_ratio":lambda x: f"{x:.3f}"}))
tab.to_csv(DATA_DIR/"Q12_top10_worst_affordability.csv", index=False)


Q11) Top-10 worst affordability (highest burger / daily GDPpc)
    country afford_ratio
    Lebanon        2.553
   Pakistan        0.195
   Honduras        0.194
  Nicaragua        0.181
     Jordan        0.119
Philippines        0.099
  Sri Lanka        0.096
      India        0.095
  Guatemala        0.094
       Peru        0.078
